In [7]:
import json
import findspark
findspark.init()

# create entry points to spark
try:
    sc.stop()
except:
    pass
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = SparkConf().setAppName("finalproject").setMaster("local[*]")
sc=SparkContext(conf = conf)
spark = SparkSession(sparkContext=sc)


In [3]:
sc

<SparkContext master=local[*] appName=finalproject>

In [229]:
with open("data/2000.json", "r") as file:
    data = file.read()
data = dict(json.loads(data))["PatentBulkData"]

print(len(data))

371


In [156]:
sampleRow = data[0]
applicantOrInventorBag = sampleRow["patentCaseMetadata"]["partyBag"]["applicantBagOrInventorBagOrOwnerBag"][1]
inventorContact = applicantOrInventorBag["inventorOrDeceasedInventor"][0]["contactOrPublicationContact"][0]
cityName = inventorContact["cityName"]
stateName = inventorContact["geographicRegionName"]["value"]
fullLocation = (cityName,stateName)
print(inventorContact)
print(fullLocation)

{u'geographicRegionName': {u'geographicRegionCategory': u'STATE', u'value': u'NY'}, u'cityName': u'NEW YORK', u'name': {u'personNameOrOrganizationNameOrEntityName': [{u'personStructuredName': {u'middleName': u'S.', u'lastName': u'NUSSENZWEIG', u'firstName': u'RUTH'}}]}, u'countryCode': u'US'}
(u'NEW YORK', u'NY')


In [157]:
inventorName = inventorContact["name"]["personNameOrOrganizationNameOrEntityName"][0]["personStructuredName"]
fullName = " ".join(inventorName.values())
print(inventorName)
print(fullName)

{u'middleName': u'S.', u'lastName': u'NUSSENZWEIG', u'firstName': u'RUTH'}
S. NUSSENZWEIG RUTH


In [158]:
fullContact = (fullName, fullLocation)
print(fullContact)

(u'S. NUSSENZWEIG RUTH', (u'NEW YORK', u'NY'))


In [159]:
#extracts the name and location of the first listed inventor.
def findNameAndLocation(data):
    for sampleRow in data:
        applicantOrInventorBag = sampleRow["patentCaseMetadata"]["partyBag"]["applicantBagOrInventorBagOrOwnerBag"][1]
        inventorContact = applicantOrInventorBag["inventorOrDeceasedInventor"][0]["contactOrPublicationContact"][0]
        cityName = inventorContact["cityName"]
        stateName = inventorContact["geographicRegionName"]["value"]
        countryCode = inventorContact["countryCode"]
        fullLocation = (cityName,stateName, countryCode)
        inventorName = inventorContact["name"]["personNameOrOrganizationNameOrEntityName"][0]["personStructuredName"]
        fullName = " ".join(inventorName.values())
        fullContact = (fullName, fullLocation)
        print(fullContact)

In [160]:
findNameAndLocation(data)

(u'S. NUSSENZWEIG RUTH', (u'NEW YORK', u'NY', u'US'))
(u' GALA JERRY', (u'FORDS', u'NJ', u'US'))
(u'P. HAUN R.', (u'EASTLAND', u'TX', u'US'))
(u' PALMERS HILBERT', (u'EINDHOVEN', u'', u'NL'))
(u'WADE LUX O.', (u'SANTA ROSA', u'CA', u'US'))
(u'L. SING EDMOND', (u'INDIANAPOLIS', u'IN', u'US'))
(u' DATE NOBUAKI', (u'KANAGAWA-KEN', u'', u'JP'))
(u'S. ATAL BISHNU', (u'NEW PROVIDENCE', u'NJ', u'US'))
(u' LOPATA IRA', (u'NEW YORK', u'NY', u'US'))
(u' HOPTNER GERD', (u'REMSCHEID', u'', u'DE'))
(u' COLE MARTIN', (u'DORKING', u'', u'GB'))
(u'A. SCHACHAR RONALD', (u'DENISON', u'TX', u'US'))
(u'J. MILANES EDDY', (u'OKLAHOMA CITY', u'OK', u'US'))
(u' NAKAJIMA HIDEKI', (u'OSAKA', u'', u'JP'))
(u'C. HARVEY JOHN', (u'NEW YORK', u'NY', u'US'))


In [291]:
def createData(year):
    with open("data/{}.json".format(year), "r") as file:
        data = file.read()
    data = dict(json.loads(data))["PatentBulkData"]
    return data

def findCompany(row):
    try:
        assignee = row["assignmentDataBag"]["assignmentData"][0]["assigneeBag"]["assignee"]
        contact = assignee[0]["contactOrPublicationContact"][0]
        name = ", ".join([x["value"] for x in contact["name"]["personNameOrOrganizationNameOrEntityName"]])
        return name
    except KeyError:
        return None

def findName(row):
        try:
            applicantOrInventorBag = row["patentCaseMetadata"]["partyBag"]["applicantBagOrInventorBagOrOwnerBag"][1]
            inventorContact = applicantOrInventorBag["inventorOrDeceasedInventor"][0]["contactOrPublicationContact"][0]
            inventorName = inventorContact["name"]["personNameOrOrganizationNameOrEntityName"][0]["personStructuredName"]
            fullName = " ".join(inventorName.values())
            return fullName
        except (KeyError, IndexError):
            return None

def findLocation(row):
        try:
            applicantOrInventorBag = row["patentCaseMetadata"]["partyBag"]["applicantBagOrInventorBagOrOwnerBag"][1]
            inventorContact = applicantOrInventorBag["inventorOrDeceasedInventor"][0]["contactOrPublicationContact"][0]
            cityName = inventorContact["cityName"]
            stateName = inventorContact["geographicRegionName"]["value"]
            countryCode = inventorContact["countryCode"]
            fullLocation = (str(cityName),str(stateName), str(countryCode))    
            return fullLocation
        except (KeyError, IndexError):
            return None

def findDate(row):
    return row["patentCaseMetadata"]["filingDate"]

for row in createData(2001):
    rowTup = (findCompany(row), findName(row), findLocation(row), findDate(row))
    rowTup = tuple(map(lambda x: str(x), rowTup))
    print(rowTup)

('INTERNATIONAL BUSINESS MACHINES CORPORATION', 'Le Pennec Jean-Francois', "('Nice', '', 'FR')", '2001-01-02')
('None', 'Allen Biglin Lance', "('Crestline', 'OH', '')", '2001-01-04')
('MUSIC.COM INC.', 'A. Bellosguardo Philippe', "('New York', 'NY', '')", '2001-01-11')
('CISCO TECHNOLOGY, INC.', 'Stephen Gress David', "('Mechanicsville', 'VA', '')", '2001-01-10')
('INTERNATIONAL BUSINESS MACHINES CORPORATION', ' Segal Alla', "('Mount Kisco', 'NY', '')", '2001-01-04')
('NEC CORPORATION', ' Shimonishi Hideyuki', "('Tokyo', '', 'JP')", '2001-01-11')
('INTERNATIONAL BUSINESS MACHINES CORPORATION', ' Kursawe Klaus', "('Rueschlikon', '', 'CH')", '2001-01-12')
('INTERNATIONAL BUSINESS MACHINES CORPORATION', 'H. Ludwig Heiko', "('Zuerich', '', 'CH')", '2001-01-05')
('INTUITIVE SURGICAL OPERATIONS, INC.', 'J. Tierney Michael', "('Pleasanton', 'CA', '')", '2001-01-12')
('MIDWEST GRAIN PRODUCTS', ' Bassi Sukh', "('Atchison', 'KS', '')", '2001-01-03')
('ESNIFF, INC.', 'P. Donahue Thomas', "('Littl